In [19]:
import numpy as np
import pandas as pd
from combination import partitions
from math import *
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score
import itertools

In [20]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [21]:
Y_id = df_test.id
df_train.drop(['id'],axis=1,inplace=True)
df_test.drop(['id'],axis=1,inplace=True)

In [22]:
input_neurons_amount, output_neurons_amount = len(df_train.columns), 2

In [23]:
def hidden_neurons_amount(alpha):
    return alpha * sqrt(input_neurons_amount * output_neurons_amount)

def condition_to_insert(partition):
    return len(partition) <= 2

In [24]:
alpha = [0.5, 2, 3]
hidden_neurons_amounts = [ceil(hidden_neurons_amount(a)) for a in alpha]
hidden_layer_sizes = []

for n in hidden_neurons_amounts:
    tuples = partitions(n, condition_to_insert)
    hidden_layer_sizes += tuples

In [25]:
params = {
    'activation': ['identity', 'logistic','tanh','relu'],
    'hidden_layer_sizes': hidden_layer_sizes,
    'solver': ['lbfgs']
}

In [26]:
gs = GridSearchCV(MLPClassifier(), params, cv=3, scoring='f1_micro', return_train_score=False)

In [27]:
Y_train = df_train.Classification
X_train = df_train.drop(['Classification'],axis=1)

In [28]:
gs.fit(X_train, Y_train);

In [29]:
best_model = gs.best_estimator_

In [30]:
Y_pred = best_model.predict(df_test)

In [31]:
results = pd.DataFrame(data={'Classification' : Y_pred, 'id' : Y_id})

In [32]:
results.to_csv('result.csv', index=False)